# City Explorer: Multi-Source Attraction & Activity Discovery with Routing

This notebook is intentionally **readable** and **step-based**.

## What it does
1. Loads required environment variables (API keys + DB path)
2. Initializes a shared SQLite database (migrations + tables)
3. Retrieves **Top 10** items for a city (ranked by review count) from:
   - Google Places (New) — tourist attractions
   - TripAdvisor Content API — attractions/activities (depending on API behavior)
4. Uses a city-level snapshot cache (`city_top10`) and an item-level cache (`item_summary`)
5. Provides an interactive UI using **ipywidgets**

## Required environment variables
- `GOOGLE_MAPS_API_KEY`
- `TRIPADVISOR_API_KEY`
- `DABN23_DB_PATH` (full file path, e.g. `G:\My Drive\dabn23_SharedDatabase\dabn23_cache.sqlite`)


In [1]:
# 0) Dependency check (optional)
# This notebook does NOT auto-install by default (cleaner + more reproducible).
AUTO_INSTALL = False

required = [
    ("requests", "requests"),
    ("pandas", "pandas"),
    ("ipywidgets", "ipywidgets"),
]

missing = []
for import_name, pip_name in required:
    try:
        __import__(import_name)
    except ImportError:
        missing.append(pip_name)

if missing:
    print("Missing packages:", ", ".join(missing))
    print("Install command:")
    print("  pip install " + " ".join(missing))
    if AUTO_INSTALL:
        import sys, subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", *missing])
        print("Installed. Re-run this cell if needed.")
else:
    print("All required packages are installed.")


All required packages are installed.


In [2]:
# 1) Make sure we can import from /src (works when running from notebooks/ folder)
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("Project root:", PROJECT_ROOT)


Project root: c:\Users\Samuel\Desktop\Git Repo\dabn23-project1\dabn23


In [3]:
# 2) Load configuration (API keys + DB path)
# config.py fails fast with a helpful error message if something is missing.

from src.config import GOOGLE_API_KEY, TA_API_KEY, DB_PATH

print("Google API key loaded (length):", len(GOOGLE_API_KEY))
print("TripAdvisor API key loaded (length):", len(TA_API_KEY))
print("DB_PATH:", DB_PATH)


Google API key loaded (length): 39
TripAdvisor API key loaded (length): 32
DB_PATH: G:\My Drive\dabn23_SharedDatabase\dabn23_places_cache.sqlite


In [4]:
# 3) Initialize the shared SQLite database (creates the file if it doesn't exist)

from pathlib import Path
from src.db import connect, migrate_if_needed, create_tables

# Ensure parent folder exists (SQLite can create the file, but not the folder)
Path(DB_PATH).parent.mkdir(parents=True, exist_ok=True)

conn = connect(DB_PATH)
migrate_if_needed(conn)   # handles legacy schemas (e.g., place_ids_json -> item_ids_json)
create_tables(conn)

tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
print("✅ DB ready. Tables:", [t[0] for t in tables])


✅ DB ready. Tables: ['city_top10', 'item_summary']


In [5]:
import src.pipelines as p
with open(p.__file__, "r", encoding="utf-8") as f:
    txt = f.read()

print("Length:", len(txt))
print(txt[:800])

Length: 5897
# src/pipelines.py
"""
Top-10 pipelines (snapshot + cache) for DABN23.

This module is meant to replace the notebook-defined pipeline functions so that:
- notebooks stay thin (just call functions)
- snapshot + caching logic lives in src/
- TripAdvisor group filtering is applied BEFORE snapshot is saved
"""

from __future__ import annotations

from typing import Any, Dict, List, Optional

from .cache import (
    get_city_snapshot_item_ids,
    save_city_snapshot_item_ids,
    get_cached_item_summary,
    upsert_item_summary,
)

from . import google_places as g
from . import tripadvisor as ta


def top10_google_attractions(
    conn,
    city: str,
    n: int = 10,
    language: str = "en",
    search_pool: int = 50,
) -> List[Dict[str, Any]]:
    """Top-N Google tourist attractions by revi


In [6]:
# Step 4 definitions with imports

from src.pipelines import top10_city

ALLOW = ["Tours", "Food & Drink", "Outdoor Activities", "Boat Tours & Water Sports", "Nightlife", "Shopping"]
DENY  = ["Sights & Landmarks", "Museums"]

def city_search(city: str):
    return top10_city(conn, city, allow_groups=ALLOW, deny_groups=DENY)

## 5) Interactive search UI (ipywidgets)

Use the controls to choose:
- city
- data source (Google or TripAdvisor)
- type (attraction/activity)

Then click **Search Top 10**.


In [7]:
from src.ui import build_city_widget

LAST_SEARCHED_CITY = None
LAST_SEARCH_RESULTS = None

build_city_widget(city_search)

{'last_city': None, 'last_results': None}

## 6) Optional: "closest two" demo (fallback)

This uses a straight-line distance fallback (Haversine) so the demo works even before
Google Routes API is integrated into `src/routing.py`.


In [8]:
from src.routing import closest_two_fallback

# Example: compute closest two among Google top-10 (needs lat/lng)
city = "Paris"
results = top10_google_attractions(city)

start = results[0]
others = results[1:]

closest = closest_two_fallback(start, others)

print("Start:", start.get("name"))
print("Closest two (fallback distance):")
for c in closest:
    print(" -", c.get("name"))


NameError: name 'top10_google_attractions' is not defined

# Selenium

## Pipelines (snapshot + cache) — Google only

In [9]:
# ============================================
# SELENIUM (NEW ENV): Google snapshot + city UI
# ============================================

import json
import sqlite3
import time
import re
import datetime

# Selenium imports
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import importlib
import inspect
import src.ui as ui
ui = importlib.reload(ui)

# ------------------------------------------------------------
# 1) Read Google snapshot names from unified DB (NEW ENV)
# ------------------------------------------------------------

def get_attraction_names(city: str, conn: sqlite3.Connection):
    """
    Look up stored Google attraction names for a city from the unified tables:
      - city_top10 (snapshot of item_ids_json)
      - item_summary (cached summaries, including name)

    Returns:
      - None if city snapshot not found
      - [] if snapshot exists but empty
      - [name, ...] in snapshot ranking order
    """
    citykey = city.strip().lower()
    source = "google"
    item_type = "attraction"

    cur = conn.cursor()

    row = cur.execute(
        """
        SELECT item_ids_json
        FROM city_top10
        WHERE city_key = ? AND source = ? AND item_type = ?
        """,
        (citykey, source, item_type),
    ).fetchone()

    if not row:
        return None

    item_ids = json.loads(row[0]) if row[0] else []
    if not item_ids:
        return []

    placeholders = ",".join("?" * len(item_ids))
    name_rows = cur.execute(
        f"""
        SELECT item_id, name
        FROM item_summary
        WHERE source = ? AND item_id IN ({placeholders})
        """,
        [source, *item_ids],
    ).fetchall()

    name_map = {pid: name for pid, name in name_rows}
    return [name_map[pid] for pid in item_ids if pid in name_map]


# ------------------------------------------------------------
# 2) Google Maps peak-hours scraping (minimal changes)
# ------------------------------------------------------------

def _parse_busy_bar(aria: str):
    """
    Parses one peak-hours bar aria-label.
    Handles English ("77% busy at 2 pm") and Swedish/Nordic ("77 aktivitet kl. 1400.").
    Returns (hour_24, pct) or None.
    """
    m = re.search(r"^(\d+)\D+?kl\.\s*(\d{2})\d{2}", aria.strip())
    if m:
        return int(m.group(2)), int(m.group(1))

    m = re.search(r"(\d+)%.*?(\d{1,2})\s*(am|pm)", aria, re.IGNORECASE)
    if m:
        pct, h, mer = int(m.group(1)), int(m.group(2)), m.group(3).lower()
        hour_24 = (h % 12) + (12 if mer == "pm" else 0)
        return hour_24, pct

    return None


def dismiss_google_consent(driver):
    """Dismiss the GDPR consent banner on Google Maps (EU only)."""
    try:
        accept_btn = WebDriverWait(driver, 8).until(
            EC.element_to_be_clickable((
                By.XPATH,
                '//button[.//span[contains(text(),"Accept all") '
                'or contains(text(),"Reject all")]]'
            ))
        )
        accept_btn.click()
        time.sleep(1)
        print("  Google consent dismissed.")
    except:
        print("  No consent popup found.")


def get_current_busyness(driver, attraction_name: str, city: str):
    """
    Searches Google Maps for attraction_name (scoped to city) and scrapes the full-day busyness.
    Returns list[int|None] with 24 entries (index = hour 0–23),
    or None if the place has no peak-hours section at all.
    """
    query = f"{attraction_name} {city}".strip()
    print(f"\n  Searching: {query}")

    driver.get("https://www.google.com/maps")
    time.sleep(2)
    dismiss_google_consent(driver)

    search_bar = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.NAME, "q"))
    )
    search_bar.clear()
    search_bar.send_keys(query)
    driver.find_element(By.CSS_SELECTOR, "button.mL3xi").click()
    time.sleep(3)

    try:
        WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "a.hfpxzc"))
        ).click()
        time.sleep(3)
        print("    Clicked top result from list.")
    except TimeoutException:
        print("    Landed directly on place page.")

    try:
        peak_section = WebDriverWait(driver, 6).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.UmE4Qe"))
        )
    except TimeoutException:
        print("    No peak hours data available.")
        return None

    hourly_data = [None] * 24
    bars = peak_section.find_elements(By.CSS_SELECTOR, "div.dpoVLd")
    print(f"    Found {len(bars)} hourly bars.")

    for bar in bars:
        aria = bar.get_attribute("aria-label") or ""

        live = re.search(r"(?:Currently|Nuvarande).*?(\d+)%", aria, re.IGNORECASE)
        if live:
            hourly_data[datetime.datetime.now().hour] = int(live.group(1))
            print(f"    ✓ Live now: {int(live.group(1))}%")
            continue

        parsed = _parse_busy_bar(aria)
        if parsed:
            hour_24, pct = parsed
            if 0 <= hour_24 <= 23:
                hourly_data[hour_24] = pct

    filled = sum(1 for x in hourly_data if x is not None)
    print(f"    ✓ Stored {filled}/24 hours.")
    return hourly_data


# ------------------------------------------------------------
# 3) Orchestrator: scrape peak-hours for the LAST searched city
# ------------------------------------------------------------

busyness_data = {}  # in-memory results

def _make_driver():
    """
    Minimal driver factory. Adjust if your environment needs a specific driver path.
    """
    # If you already configure options elsewhere, replace this with your setup.
    return webdriver.Chrome()

def scrape_peak_hours(city: str, conn: sqlite3.Connection):
    """
    Uses Google snapshot names (from unified DB) and scrapes peak-hours from Google Maps.
    Stores into global busyness_data[city].
    """
    scraped_at = datetime.datetime.now().strftime("%H:%M")
    city_key = city.strip()

    print(f"Looking up Google snapshot names for '{city_key}'...")
    names = get_attraction_names(city_key, conn)

    if names is None:
        print(f"  ✗ No snapshot found for '{city_key}'. Click Search Top 10 first.")
        return
    if not names:
        print(f"  ✗ Snapshot exists but empty for '{city_key}'.")
        return

    driver = _make_driver()
    attractions = {}

    try:
        for name in names:
            hourly = get_current_busyness(driver, name, city_key)
            attractions[name] = hourly
            time.sleep(2)
    finally:
        driver.quit()
        print("\nDriver closed.")

    busyness_data[city_key] = {
        "scraped_at": scraped_at,
        "attractions": attractions,
    }

    now_hour = datetime.datetime.now().hour
    print(f"\n{'='*54}")
    print(f"  CURRENT BUSYNESS — {city_key}  (scraped at {scraped_at})")
    print(f"{'='*54}")
    for name, hourly in attractions.items():
        if hourly is None:
            val = "N/A (no GM data)"
        elif hourly[now_hour] is None:
            val = "N/A (no data this hour)"
        else:
            val = f"{hourly[now_hour]}%"
        print(f"  {name:<44} {val}")


# ------------------------------------------------------------
# 4) Widget integration: store last searched city (NO extra button)
# ------------------------------------------------------------
# We wrap the existing ui.build_city_widget so that it returns a state dict.
# You only click "Search Top 10". Later, call run_selenium_last().

def build_city_widget_with_state(search_fn, *, default_city="Paris", title=None):
    """
    Uses src.ui's helpers but stores last_city + last_results for later use.
    Returns state dict: {"last_city": ..., "last_results": ...}
    """
    import ipywidgets as widgets
    from IPython.display import display

    if title:
        display(widgets.HTML(f"<b>{title}</b>"))

    city_input = widgets.Text(
        value=default_city,
        description="City:",
        placeholder="e.g., Paris, Rome, Stockholm",
        layout=widgets.Layout(width="420px"),
    )

    btn_search = widgets.Button(description="Search Top 10", button_style="primary")
    output = widgets.Output()

    state = {"last_city": None, "last_results": None}

    def on_search(_):
        with output:
            output.clear_output()

            city = city_input.value.strip()
            if not city:
                print("Please enter a city name.")
                return

            state["last_city"] = city
            print(f"Searching Top 10 | city='{city}'\n")

            try:
                results = search_fn(city)
                state["last_results"] = results

                if not results:
                    print("No results found.")
                    return

                display(ui.results_to_dataframe(results))

            except Exception as e:
                print("Error:", str(e))

    btn_search.on_click(on_search)

    display(widgets.VBox([
        widgets.HBox([city_input, btn_search]),
        output,
    ]))

    return state


# Call this after you used the widget at least once
def run_selenium_last(state):
    """
    Runs selenium for the last city searched in the widget.
    """
    city = state.get("last_city")
    if not city:
        print("No city stored yet. Click 'Search Top 10' first.")
        return
    scrape_peak_hours(city, conn)


print("Ready. Next: create the widget with state = build_city_widget_with_state(city_search, ...)")

Ready. Next: create the widget with state = build_city_widget_with_state(city_search, ...)


In [10]:
def run_selenium_for_last_city():
    if not LAST_SEARCHED_CITY:
        print("No city stored yet. Click 'Search Top 10' first.")
        return
    scrape_peak_hours(LAST_SEARCHED_CITY, conn)

In [12]:
print(LAST_SEARCHED_CITY)

None


In [11]:
run_selenium_for_last_city(state)

NameError: name 'state' is not defined